In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

# Find project root (folder containing "data")
cwd = Path.cwd().resolve()

PROJECT_ROOT = None
for p in [cwd] + list(cwd.parents):
    if (p / "data").exists():
        PROJECT_ROOT = p
        break

if PROJECT_ROOT is None:
    raise FileNotFoundError(
        "Could not locate project root. "
        "Ensure the notebook is inside the project folder containing 'data/'."
    )

# Data directories
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
INTERIM_DIR = DATA_DIR / "interim"

# Raw files
PRIOR_CSV = RAW_DIR / "order_products__prior.csv"
TRAIN_CSV = RAW_DIR / "order_products__train.csv"
ORDERS_CSV = RAW_DIR / "orders.csv"
PRODUCTS_CSV = RAW_DIR / "products.csv"

# Feature outputs
PRIOR_PARQUET = INTERIM_DIR / "prior.parquet"
TRAIN_PARQUET = INTERIM_DIR / "train.parquet"
FEATURES_PARQUET = INTERIM_DIR / "feature_processed.parquet"

print("Project root:", PROJECT_ROOT)

Project root: C:\Users\User\instacart-retail-project


In [2]:
# Load Instacart Tables (Prediction)

print("Loading Instacart raw tables")

orders = pd.read_csv(ORDERS_CSV)
prior = pd.read_csv(PRIOR_CSV)
train = pd.read_csv(TRAIN_CSV)
products = pd.read_csv(PRODUCTS_CSV)

print("orders:", orders.shape)
print("prior:", prior.shape)
print("train:", train.shape)
print("products:", products.shape)

Loading Instacart raw tables
orders: (3421083, 7)
prior: (32434489, 4)
train: (1384617, 4)
products: (49688, 4)


In [3]:
# combine User & Order Info to Prior

print("Merging prior with orders")

prior = prior.merge(
    orders[["order_id", "user_id", "order_number"]],
    on="order_id",
    how="left"
)

print("prior shape after merge:", prior.shape)
prior.head()

Merging prior with orders
prior shape after merge: (32434489, 6)


,order_id,product_id,add_to_cart_order,reordered,user_id,order_number
0,2,33120,1,1,202279,3
1,2,28985,2,1,202279,3
2,2,9327,3,0,202279,3
3,2,45918,4,1,202279,3
4,2,30035,5,0,202279,3


In [4]:
# User Features


print("Building user-level features")

user_features = (
    prior.groupby("user_id")
    .agg(
        user_total_orders=("order_number", "max"),
        user_total_items=("product_id", "count"),
        user_distinct_products=("product_id", "nunique")
    )
    .reset_index()
)

print("user_features shape:", user_features.shape)
user_features.head()

Building user-level features
user_features shape: (206209, 4)


,user_id,user_total_orders,user_total_items,user_distinct_products
0,1,10,59,18
1,2,14,195,102
2,3,12,88,33
3,4,5,18,17
4,5,4,37,23


Feature Extraction Setup (Cells 1–4)

This section prepares the Instacart transaction data for feature engineering by loading historical purchases and constructing user-level behavioral summaries. The goal is to build predictive features using only past customer behavior (prior orders), ensuring a correct temporal setup for reorder prediction.

---

Instacart Data Structure

The Instacart dataset divides orders into three groups:

- prior → historical orders (customer purchase history)
- train → most recent known order (prediction target)
- test → future order (not used in this project)

For feature construction, only prior orders are used because they represent information available before the prediction point.
Train orders are reserved for label creation in later steps.

---

Cell 1 — Project Path Configuration

The notebook first locates the project root directory by searching for the folder that contains the "data/" directory. This allows all file paths to be defined relative to the project structure rather than using hard-coded absolute paths.

This ensures:

- portability across team members’ machines
- consistent access to "data/raw" and "data/interim"
- reproducible notebook execution

After locating the project root, paths to the raw Instacart CSV files and interim feature outputs are defined.

---

Cell 2 — Load Instacart Tables

The core Instacart tables are loaded from "data/raw":

- "orders" → order metadata (user, order number, split)
- "prior" → historical product purchases
- "train" → products in the last observed order
- "products" → product catalog

The shapes confirm correct loading:

- ~3.4M orders
- ~32.4M prior purchases
- ~1.38M train purchases
- ~49k products

The prior and train tables are intentionally kept separate to preserve the temporal boundary between historical behavior and prediction targets.

---

Cell 3 — Merge Prior Purchases with Orders

The "prior" purchase table originally contains only order-level information.
To compute user-level behavioral features, each purchase must be linked to:

- the customer ("user_id")
- the sequence of the order ("order_number")

This is achieved by merging prior purchases with the "orders" table on "order_id".

After this merge, each purchase record represents:

(user_id, product_id, order_number)

This structure converts transaction logs into user purchase histories, which are required for feature aggregation.

Only prior orders are merged, ensuring that features are derived strictly from historical data and preventing information leakage from future (train) orders.

---

Cell 4 — User-Level Features

User-level behavioral summaries are computed from prior purchases by grouping transactions by "user_id".

For each customer, the following features are derived:

- user_total_orders → number of historical orders placed
- user_total_items → total items purchased
- user_distinct_products → number of unique products purchased

These features describe overall shopping intensity and product variety for each customer.

They will later be used to normalize user–product interactions (e.g., purchase frequency relative to total orders).

The resulting table contains one row per user (~206k users), representing aggregated historical behavior.

---

Summary of Progress

After Cells 1–4:

- Historical purchases are isolated (prior only)
- Transactions are linked to users and order sequence
- Customer-level behavioral features are constructed

The dataset is now ready for product-level and user–product interaction feature engineering.

In [6]:
# Product Features


print("Building product-level features")

product_features = (
    prior.groupby("product_id")
    .agg(
        prod_total_purchases=("user_id", "count"),
        prod_distinct_users=("user_id", "nunique")
    )
    .reset_index()
)

print("product_features shape:", product_features.shape)
product_features.head()

Building product-level features
product_features shape: (49677, 3)


,product_id,prod_total_purchases,prod_distinct_users
0,1,1852,716
1,2,90,78
2,3,277,74
3,4,329,182
4,5,15,6


In [7]:
# User–Product Interaction Features


print("Building user–product interaction features")

user_product_features = (
    prior.groupby(["user_id", "product_id"])
    .agg(
        up_purchase_count=("order_number", "count"),
        up_first_order=("order_number", "min"),
        up_last_order=("order_number", "max")
    )
    .reset_index()
)

print("user_product_features shape:", user_product_features.shape)
user_product_features.head()

Building user–product interaction features
user_product_features shape: (13307953, 5)


,user_id,product_id,up_purchase_count,up_first_order,up_last_order
0,1,196,10,1,10
1,1,10258,9,2,10
2,1,10326,1,5,5
3,1,12427,10,1,10
4,1,13032,3,2,10


Product and User–Product Feature Engineering (Cells 5–6)

This stage constructs product-level and user–product interaction features from historical (prior) purchases.
These features capture both global product popularity and individual customer purchase behavior, which are key signals for reorder prediction.

---

Cell 5 — Product-Level Features

Product-level statistics are computed by aggregating prior purchases by "product_id".
These features describe how frequently each product is purchased across the entire customer base.

Computed features:

- prod_total_purchases → total number of times the product was purchased
- prod_distinct_users → number of unique customers who purchased the product

Interpretation:
Each row represents a single product with its global popularity and adoption level.

The resulting table contains approximately 49.6k products, corresponding to products that appear in historical (prior) orders.
Products that appear only in train/test orders are excluded, which is correct because features must be derived solely from past behavior.

These features help distinguish between:

- Products that are popular for everyone
- Products that are specific to certain users

This distinction is important for reorder modeling, since frequent purchases of globally popular items carry different meaning than frequent purchases of niche items.

---

Cell 6 — User–Product Interaction Features

User–product interaction features are computed by aggregating prior purchases by both "user_id" and "product_id".
These features capture each customer's personal purchase history for individual products.

Computed features:

- up_purchase_count → number of times the user purchased the product
- up_first_order → order number when the product was first purchased
- up_last_order → order number when the product was most recently purchased

Interpretation:
Each row represents a (user, product) pair summarizing that user's interaction history with the product.

These features capture three core behavioral signals:

- Frequency → how often the user buys the product
- Loyalty → how early the product entered the user's basket
- Recency → how recently the user bought the product

The resulting table contains approximately 13 million user–product pairs, representing all products purchased by each user in prior orders.

---

Role in Reorder Prediction

Cells 5–6 establish the two most important components of behavioral modeling:

- Product popularity (global behavior)
- User–product history (personal behavior)

These features will later be combined with user-level features to construct normalized predictors such as:

- User purchase rate for a product
- Product importance for a user
- Recency-based reorder likelihood

After Cells 5–6, the pipeline has captured both global product behavior and individual customer purchase history from prior orders.

In [8]:
# Merge User, Product, and User–Product Features


print("Merging feature blocks")

features = (
    user_product_features
    .merge(user_features, on="user_id", how="left")
    .merge(product_features, on="product_id", how="left")
)

print("Final feature table shape:", features.shape)
features.head()

Merging feature blocks
Final feature table shape: (13307953, 10)


,user_id,product_id,up_purchase_count,up_first_order,up_last_order,user_total_orders,user_total_items,user_distinct_products,prod_total_purchases,prod_distinct_users
0,1,196,10,1,10,10,59,18,35791,8000
1,1,10258,9,2,10,10,59,18,1946,557
2,1,10326,1,5,5,10,59,18,5526,1923
3,1,12427,10,1,10,10,59,18,6476,1679
4,1,13032,3,2,10,10,59,18,3751,1286


Feature Integration (Cell 7)

This step merges the previously constructed feature blocks into a single modeling dataset.
The goal is to create a unified feature table where each row represents a (user, product) pair enriched with behavioral and popularity attributes.

---

Feature Sources

Three feature tables are combined:

- User features → overall customer shopping behavior
- Product features → global product popularity
- User–product features → individual interaction history

The merge is performed starting from the user–product table, since it defines all valid user–product interactions observed in historical data.

---

Resulting Feature Structure

After merging, each row contains:

- "user_id"
- "product_id"
- user-level features
- product-level features
- user–product interaction features

This produces the complete behavioral representation needed for reorder prediction.

---

Why This Merge Is Correct

The merge order preserves the interaction structure:

- Start with user–product pairs
- Attach user attributes via "user_id"
- Attach product attributes via "product_id"

This ensures that each feature row corresponds to an actual historical purchase relationship.

---

Role in the Prediction Pipeline

After Cell 7, the dataset now contains:

- customer behavior
- product popularity
- interaction frequency and recency

These combined features form the core input for modeling reorder probability.

The next stage will derive normalized predictors (e.g., purchase rates and recency gaps) from these merged attributes.

In [9]:
# Derived Predictive Features

print("Creating derived predictive features")

features["up_order_rate"] = features["up_purchase_count"] / features["user_total_orders"]
features["up_item_rate"] = features["up_purchase_count"] / features["user_total_items"]
features["up_recency"] = features["user_total_orders"] - features["up_last_order"]
features["up_product_share"] = features["up_purchase_count"] / features["prod_total_purchases"]

print("Derived features added.")
features.head()

Creating derived predictive features
Derived features added.


,user_id,product_id,up_purchase_count,up_first_order,up_last_order,user_total_orders,user_total_items,user_distinct_products,prod_total_purchases,prod_distinct_users,up_order_rate,up_item_rate,up_recency,up_product_share
0,1,196,10,1,10,10,59,18,35791,8000,1.0,0.169492,0,0.000279
1,1,10258,9,2,10,10,59,18,1946,557,0.9,0.152542,0,0.004625
2,1,10326,1,5,5,10,59,18,5526,1923,0.1,0.016949,5,0.000181
3,1,12427,10,1,10,10,59,18,6476,1679,1.0,0.169492,0,0.001544
4,1,13032,3,2,10,10,59,18,3751,1286,0.3,0.050847,0,0.000800


Derived Predictive Features (Cell 8)

This step transforms aggregated behavioral statistics into normalized predictors that directly capture reorder likelihood.
While previous features describe counts and totals, derived features express relative importance, frequency, and recency, which are stronger indicators of future purchases.

---

Derived Features

The following normalized predictors are computed:

- up_order_rate
  Number of times the user purchased the product divided by the user’s total orders.
  Represents how frequently the product appears in the user’s shopping routine.

- up_item_rate
  Number of times the user purchased the product divided by the user’s total items purchased.
  Measures the share of the user’s basket attributable to the product.

- up_recency
  Difference between the user’s last order number and the most recent order in which the product was purchased.
  Indicates how recently the user bought the product.

- up_product_share
  Number of times the user purchased the product divided by the product’s total purchases across all users.
  Measures how important the user is relative to the product’s global popularity.

---

Behavioral Interpretation

These features capture the main behavioral drivers of reorder probability:

- Products bought frequently are likely to be reordered
- Recently purchased products are more likely to reappear
- Products that dominate a user’s basket indicate strong preference
- Products disproportionately purchased by a user indicate loyalty

---

Role in the Modeling Pipeline

After Cell 8, the dataset now contains normalized behavioral signals derived from:

- user behavior
- product popularity
- user–product interaction

These predictors form the core inputs for machine learning models estimating reorder probability.

In [10]:
# Create Reorder Labels

print("Creating labels from train orders")

# Attach user_id to train purchases
train_orders = orders[orders["eval_set"] == "train"][["order_id", "user_id"]]

train_labels = (
    train.merge(train_orders, on="order_id", how="left")
    [["user_id", "product_id"]]
)

train_labels["label"] = 1

print("train_labels shape:", train_labels.shape)
train_labels.head()

Creating labels from train orders
train_labels shape: (1384617, 3)


,user_id,product_id,label
0,112108,49302,1
1,112108,11109,1
2,112108,10246,1
3,112108,49683,1
4,112108,43633,1


Label Construction from Train Orders (Cell 9)

This step creates the target variable for reorder prediction by identifying which products were purchased in each user’s final observed order.

---

Instacart Temporal Structure

The Instacart dataset separates orders into:

- prior → historical orders used for feature construction
- train → most recent known order used as prediction target

For each user, the train order represents the next basket following all prior purchases.

---

Label Definition

Products appearing in a user’s train order are considered reordered items relative to prior history.

The label is therefore defined as:

- label = 1 → product purchased in the train order
- label = 0 → product not purchased in the train order

---

Construction Process

1. Identify all orders where "eval_set = train"
2. Attach "user_id" to train purchases
3. Create a "(user_id, product_id)" table of reordered items
4. Assign label value "1" to these pairs

This produces the ground-truth reorder events for supervised learning.

---

Role in the Prediction Pipeline

After Cell 9, the project has:

- historical features from prior orders
- true reorder outcomes from train orders

These labels will be merged with the feature table so that each user–product pair can be classified as reordered or not reordered.

In [11]:
# Merge Labels with Feature Table

print("Merging labels into feature table")

features = features.merge(
    train_labels,
    on=["user_id", "product_id"],
    how="left"
)

# Products not in train order → label 0
features["label"] = features["label"].fillna(0).astype(int)

print("Final dataset shape:", features.shape)
features.head()

Merging labels into feature table
Final dataset shape: (13307953, 15)


,user_id,product_id,up_purchase_count,up_first_order,up_last_order,user_total_orders,user_total_items,user_distinct_products,prod_total_purchases,prod_distinct_users,up_order_rate,up_item_rate,up_recency,up_product_share,label
0,1,196,10,1,10,10,59,18,35791,8000,1.0,0.169492,0,0.000279,1
1,1,10258,9,2,10,10,59,18,1946,557,0.9,0.152542,0,0.004625,1
2,1,10326,1,5,5,10,59,18,5526,1923,0.1,0.016949,5,0.000181,0
3,1,12427,10,1,10,10,59,18,6476,1679,1.0,0.169492,0,0.001544,0
4,1,13032,3,2,10,10,59,18,3751,1286,0.3,0.050847,0,0.000800,1


Final Supervised Dataset Construction (Cell 10)

This step merges the reorder labels derived from train orders into the feature table constructed from prior purchases.
The result is a supervised dataset where each user–product pair is associated with both predictive features and the observed reorder outcome.

---

Merge Logic

The feature table contains all historical "(user_id, product_id)" interactions derived from prior orders.
The label table contains only products purchased in each user’s train order.

A left merge is performed on "(user_id, product_id)":

- If the product appears in the train order → label = 1
- If not → label = 0

Missing labels after the merge therefore correspond to products not reordered in the final order and are assigned zero.

---

Resulting Dataset Structure

Each row now represents a historical user–product relationship enriched with:

- user behavioral features
- product popularity features
- interaction frequency and recency
- reorder label

This produces a complete supervised learning dataset for reorder prediction.

---

Interpretation

The dataset now answers the learning question:

«Given a user’s historical purchases and product behavior, will the user reorder this product in their next order?»

---

Role in the Pipeline

After Cell 10, the project has produced the final modeling dataset containing:

- engineered features from prior orders
- ground-truth reorder outcomes from train orders

This dataset is ready for machine learning model training and evaluation.

In [12]:
# Save Final Feature Dataset

print("Saving feature dataset")

INTERIM_DIR.mkdir(parents=True, exist_ok=True)
features.to_parquet(FEATURES_PARQUET, index=False)

print("Saved to:", FEATURES_PARQUET)
print("Final shape:", features.shape)

Saving feature dataset
Saved to: C:\Users\User\instacart-retail-project\data\interim\feature_processed.parquet
Final shape: (13307953, 15)


Feature Dataset Persistence (Cell 11)

This step saves the fully engineered supervised dataset to the project’s interim data directory for reuse in downstream modeling and analysis.

---

Purpose

Feature engineering is computationally intensive, especially for large transactional datasets.
Persisting the processed feature table avoids repeated recomputation and ensures that all team members and pipeline stages use a consistent dataset.

---

Output Dataset

The saved dataset contains:

- user identifiers
- product identifiers
- user behavioral features
- product popularity features
- user–product interaction features
- reorder label

Each row represents a historical user–product pair annotated with whether the product was reordered in the user’s next order.

---

Storage Location

The dataset is written to:

data/interim/feature_processed.parquet

The interim directory stores derived datasets that are ready for modeling but not yet final analytical outputs.

---

Role in the Project Pipeline

After this step, the feature extraction stage is complete.
The saved dataset becomes the input for:

- machine learning model training
- evaluation experiments
- dashboard insights
- team collaboration

This separation between raw data and engineered features supports reproducibility and modular project design.

## Reorder Prediction Modeling

This section trains and evaluates a machine learning model to predict whether a user will reorder a product in their next order.  
The model uses the engineered behavioral features constructed from prior purchases.

In [4]:
# Load Feature Dataset
print("Loading feature dataset")

features = pd.read_parquet(FEATURES_PARQUET)

print("Shape:", features.shape)
features.head()

Loading feature dataset
Shape: (13307953, 15)


,user_id,product_id,up_purchase_count,up_first_order,up_last_order,user_total_orders,user_total_items,user_distinct_products,prod_total_purchases,prod_distinct_users,up_order_rate,up_item_rate,up_recency,up_product_share,label
0,1,196,10,1,10,10,59,18,35791,8000,1.0,0.169492,0,0.000279,1
1,1,10258,9,2,10,10,59,18,1946,557,0.9,0.152542,0,0.004625,1
2,1,10326,1,5,5,10,59,18,5526,1923,0.1,0.016949,5,0.000181,0
3,1,12427,10,1,10,10,59,18,6476,1679,1.0,0.169492,0,0.001544,0
4,1,13032,3,2,10,10,59,18,3751,1286,0.3,0.050847,0,0.000800,1


In [5]:
# Train/Validation Split

from sklearn.model_selection import GroupShuffleSplit

print("Creating train/validation split by user...")

X = features.drop(columns=["label"])
y = features["label"]
groups = features["user_id"]

splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(splitter.split(X, y, groups=groups))

X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)

Creating train/validation split by user...
Train shape: (10627766, 14)
Validation shape: (2680187, 14)


The resulting split produced approximately 10.6 million training rows and 2.68 million validation rows. Because the split is performed at the user level rather than the row level, the exact proportion of rows may deviate slightly from 80/20 due to variation in the number of products purchased by each user. This behavior is expected and ensures that each user appears in only one subset.

In [6]:
# Train Random Forest Model

from sklearn.ensemble import RandomForestClassifier

print("Training Random Forest model")

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_leaf=5,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

rf_model.fit(
    X_train.drop(columns=["user_id", "product_id"]),
    y_train
)

print("Model training complete.")

Training Random Forest model
Model training complete.


## Random Forest Training

A Random Forest classifier is trained to predict whether a user will reorder a product in their next order.  
Random Forest is well suited to behavioral transaction data because it captures nonlinear relationships and interactions between features without requiring feature scaling.

The model is trained using all engineered behavioral predictors while excluding identifier columns (`user_id`, `product_id`), which do not represent intrinsic behavioral information and could lead to overfitting.

Class imbalance is addressed using balanced class weights, since reordered products represent a minority of observations in the dataset.  
The model uses 200 trees to provide stable ensemble predictions while remaining computationally feasible for the large dataset size.

This model learns patterns relating purchase frequency, recency, and product importance to reorder probability.

In [7]:
# Validation Evaluation

from sklearn.metrics import roc_auc_score, average_precision_score

print("Evaluating model on validation set")

X_val_model = X_val.drop(columns=["user_id", "product_id"])
val_probs = rf_model.predict_proba(X_val_model)[:, 1]

roc_auc = roc_auc_score(y_val, val_probs)
pr_auc = average_precision_score(y_val, val_probs)

print("ROC AUC:", roc_auc)
print("PR AUC:", pr_auc)

Evaluating model on validation set
ROC AUC: 0.7956465065132405
PR AUC: 0.23068975174499257


### Conclusion

This notebook implemented an end-to-end machine learning pipeline to predict product reorder behavior in the Instacart dataset.
Historical transaction data from prior orders was transformed into user-level, product-level, and user–product interaction features capturing key behavioral signals such as purchase frequency, recency, and product importance.

A supervised dataset was constructed by linking these engineered features with reorder labels derived from each user’s final observed (train) order.
To ensure realistic evaluation and prevent information leakage, the dataset was split by user so that all interactions of a given customer appeared in only one subset.

A Random Forest classifier was trained on the engineered behavioral features and evaluated on unseen users.
The model achieved a ROC AUC of approximately 0.80 and a PR AUC of approximately 0.23, indicating strong ability to distinguish reordered from non-reordered products and good precision–recall performance for this highly imbalanced retail prediction task.

Feature importance analysis showed that predictors related to purchase frequency and recency were the dominant drivers of reorder probability, confirming that the feature engineering approach successfully captured meaningful customer purchasing patterns.

Overall, the results demonstrate that historical purchase behavior provides reliable signals for predicting future product reorders.
Such predictive models can support data-driven retail applications including personalized recommendations, targeted promotions, and demand forecasting within the broader Instacart analytics project.

In [9]:
# Export Dashboard Predictions (with names)

# reload products table
products = pd.read_csv(PRODUCTS_CSV)

export_cols = [
    "user_id",
    "product_id",
    "up_order_rate",
    "up_recency",
    "prod_total_purchases"
]

dashboard_df = X_val[export_cols].copy()
dashboard_df["reorder_prob"] = val_probs

# add product names
dashboard_df = dashboard_df.merge(
    products[["product_id", "product_name"]],
    on="product_id",
    how="left"
)

dashboard_path = PROJECT_ROOT / "data" / "processed" / "reorder_predictions.csv"
dashboard_df.to_csv(dashboard_path, index=False)

print("Dashboard predictions saved to:", dashboard_path)
print("Rows:", dashboard_df.shape[0])

Dashboard predictions saved to: C:\Users\User\instacart-retail-project\data\processed\reorder_predictions.csv
Rows: 2680187


In [10]:
dashboard_df.head()

,user_id,product_id,up_order_rate,up_recency,prod_total_purchases,reorder_prob,product_name
0,5,3376,0.25,0,12738,0.634050,Organic Whole Kernel Sweet Corn No Salt Added
1,5,5999,0.25,0,106,0.257924,"Semi-Soft Cheese, Ripened"
2,5,6808,0.25,2,575,0.273992,Organic Spinach & Cheese Tortellini
3,5,8518,0.50,0,66986,0.431552,Organic Red Onion
4,5,11777,1.00,0,26220,0.691633,Red Raspberries
